THIS IS TEST
DO NOT CONDEMN
just tried to preprocess data

In [24]:
import pandas as pd
import numpy as np
import seaborn as sns

In [25]:
domains_df = pd.read_csv('/home/ostgot/Desktop/research/BIG_data/graph_thing/real_estate_domains.tsv', sep=('\t'))
domains_df.head(20)

,domain
0,vk.com/public174839173
1,t.me/tweed_realtyy
2,instagram.com/tweedrealty
3,twitter.com/tweedru
4,youtube.com/channel/uca1z9uco37tm_yp9fi49rfq
5,tweed.ru
6,facebook.com/tweed.ru
7,vk.com/tweed_realty
8,pskovzhilstroi.ru
9,vk.com/domapskov


Test sample is a sample of 1000 random urls from dataframe

In [31]:
test_domains = domains_df.sample(5000).domain
test_domains[:5]

15278            vk.com/red_220
6097       standart-trading.com
53669    vk.com/broker_partnery
50312      poselok-exclusive.ru
6730                daniels.com
Name: domain, dtype: object

In [32]:
test_domains.nunique()

4523

In [33]:
test_domains.drop_duplicates(inplace=True)
test_domains.nunique()

4523

In [34]:
# based on hand search :)
bad_starts = 'instagram facebook linkedin twitter youtube vk t wa ok viber plus zen'.split()

In [50]:
def check_if_needless(url):
    start = url.split('.')[0]
    if start in bad_starts or '%' in start:
        return None
    else:
        return 'https://' + url

# TODO: multiprocess it
processed_domains = test_domains.apply(check_if_needless).dropna()
print(processed_domains.nunique())
processed_domains

2106


6097     https://standart-trading.com
50312    https://poselok-exclusive.ru
6730              https://daniels.com
74447      https://etalongroup.ru/spb
54475              https://domlayt.ru
                     ...             
12459            https://azino777.com
45119         https://pereplanhome.ru
55365       https://law-laboratory.ru
46567         https://capitalgroup.ru
56788                 https://hm-d.ru
Name: domain, Length: 2106, dtype: object

In [51]:
import re

def extract_main_url(s):
    m = re.search('https?://([A-Za-z_0-9.-]+).*', s)
    if m:
        return 'https://'+m.group(1)
    
processed_domains = processed_domains.apply(extract_main_url)
processed_domains.nunique()

1848

In [52]:
import requests

def url_checker(url):
    try:
        get = requests.get(url)
        if get.status_code == 200:
            return url
        else: 
            return ''
    except:
        # TODO: print exception type, it might help with analyzing
        print(url + ' is bad')
        return ''

https://azino777.com is bad
https://zblife.ru is bad


In [40]:
from concurrent.futures import ThreadPoolExecutor

with ThreadPoolExecutor(32) as executor:
    results = executor.map(url_checker, processed_domains)

%%time

https://azimut21.ru is bad
https://dsinv.ru is bad
https://kvartiravklinu.ru is bad
https://promupravlenie.ru is bad
https://baltzem.ru/kottedzhnyy-poselok-gluhovo is bad
https://bestrm.ru is bad
https://m2dm.ru is bad
https://lsr.ru/spb/kontakty/regionalnye-ofisy is bad
https://chernavski.ru is bad
https://akvilon-invest.ru/objects/postroennye/zhk-uyutnyy is bad
https://bikland.ru is bad
https://glazkova23.ru is bad
https://m.sz-dinasty.ru is bad
https://smu3.org is bad
https://ktm62.ru is bad
https://incompany.ru is bad
https://yarmarka-tula.ru is bad
https://abakan.superxata.ru is bad
https://edinstvo62.ru/building/kvartiry-v-centre is bad
https://anstel.ru/buildings/under-construction/zhiloy-dom-grani-1-pozitsiya is bad
https://kurganselstroy.ru/doma is bad
https://myzu.tb.ru is bad
https://izhevsk.kvartalrf.ru is bad
https://arenda.avtogorod.su is bad
https://pgi.su/opisanie-zk-zhemchuzhina-serpuxov is bad
https://good-zem.com/poselok/posyolok-kakuzevo-park is bad
https://arch174.

KeyboardInterrupt: 

In [ ]:
processed_domains = np.unique(results)[1:]
len(processed_domains)

245

THE END

In [ ]:
ready_domains = pd.Series(processed_domains)
ready_domains[:5]

0                  https://1470umto.ru
1                    https://23city.ru
2                 https://abc-sklad.ru
3                 https://absrealty.ru
4    https://aeroflotovec-2.landorg.ru
dtype: object

In [ ]:
ready_domains.to_csv('all_working_domains.csv', index=False)

In [ ]:
import requests
import re
from bs4 import BeautifulSoup as bs
import numpy as np
import pandas as pd

In [ ]:
URL = 'https://c-ns.ru'

PHONE_PATTERNS = [
    r'\(\d{4}\) \d{3}-\d{3}',
    r'\(\d{4}\) \d{2}-\d{2}-\d{2}',
    
    # 1 3 3 (2 2 | 4)
    r'\(?\+?[78][\s-]?\(?\d{3}\)?[\s-]?\d{3}[\s-]?\d{2}[\s-]?\d{2}',

    # 1|0 4 2 (2 2 | 4)
    r'\+\d?[\s-]?\(?\d{4}\)?[\s-]\d{2}[\s-]\d{2}[\s-]?\d{2}',
    
    # 3 2 2
    r'\(?\d{3}\)?[\s-]\d{2}[\s-]?\d{2}',
    
    # 3 1 3
    r'\(?\d{3}\)?[\s-]\d{1}[\s-]?\d{3}',
    
    # 3 3 1 3
    r'\+[][\s-]?\(?\d{3}\)?[\s-]?\d{3}[\s-]?\d{1}[\s-]?\d{3}',
]

for p in PHONE_PATTERNS:
    print(re.findall(p, '+7 (343) 289-02-59'))

In [ ]:
def extract_mail(line):
    match = re.search(r'[\w.+-]+@[\w-]+\.[\w.-]+', line)
    return match.group(0)

def get_phone_numbers(line):
    global PHONE_PATTERNS
    for pattern in PHONE_PATTERNS:
        if len(re.findall(pattern, line)) > 0:
            return tuple(np.unique(re.findall(pattern, line)))
        

def get_mails(text):
    if '@' in text:
        try:
            return extract_mail(text)
        except:
            pass
        

In [ ]:
urls = pd.read_csv('all_working_domains.csv', header=None)
urls = urls[urls.columns[0]]
urls[:5]

In [ ]:
def parse_one_url(url):
    req = requests.get(url)
    soup = bs(req.text, 'html.parser')
    texts = soup.find_all(text=True)
    texts = u" ".join(t.strip() for t in texts)
    l = [url,  get_phone_numbers(texts), get_mails(texts)];
    
    return l

In [ ]:
from multiprocessing import Pool

# This thing is pretty fast
with Pool(processes=128) as mp_pool:
    result = mp_pool.map(parse_one_url, urls)
    
result

In [ ]:
df = pd.DataFrame(result, columns =['url', 'tel', 'mail'], dtype = float)
df


In [ ]:
def standart_phone(t):
    if t != None:
        res = []
        for p in t:
            r = ''
            for c in p:
                if c.isdigit():
                    r+=c
            res.append(r) 
        return tuple(np.unique(res))
    else:
        return None
           
    
df.tel = df.tel.apply(standart_phone)
df